# Tworzymy nazwane klastry

W lekcji tej użyjemy LLM-ów do pomocy przy znalezieniu odpowiednich nazw i opisów dla klastrów.

In [1]:
import json
import pandas as pd
from openai import OpenAI
from dotenv import dotenv_values
from pycaret.clustering import predict_model, load_model

In [2]:
env = dotenv_values(".env")

openai_client = OpenAI(api_key=env["OPENAI_API_KEY"])

ładujemy nasze dane

In [3]:
df = pd.read_csv('welcome_survey_simple_v2.csv', sep=';')
df.head()

,age,edu_level,fav_animals,fav_place,gender
0,<18,Podstawowe,Brak ulubionych,NaN,Kobieta
1,25-34,Średnie,Psy,Nad wodą,Mężczyzna
2,45-54,Wyższe,Psy,W lesie,Mężczyzna
3,35-44,Średnie,Koty,W górach,Mężczyzna
4,35-44,Wyższe,Psy,Nad wodą,Mężczyzna


ładujemy model wytrenowany w poprzedniej lekcji

In [4]:
kmeans_pipeline = load_model('welcome_survey_clustering_pipeline_v2')

Transformation Pipeline and Model Successfully Loaded


aplikujemy model do danych

In [5]:
df_with_clusters = predict_model(model=kmeans_pipeline, data=df)
df_with_clusters["Cluster"].value_counts()

Cluster 4    41
Cluster 1    38
Cluster 3    32
Cluster 2    30
Cluster 5    28
Cluster 6    28
Cluster 0    22
Cluster 7    10
Name: Cluster, dtype: int64

stworzymy teraz prompt, który prześlemy do LLM-a w celu znalezienia odpowiednich nazw i opisów dla klastrów

In [6]:
cluster_descriptions = {}
for cluster_id in df_with_clusters['Cluster'].unique():
    cluster_df = df_with_clusters[df_with_clusters['Cluster'] == cluster_id]
    summary = ""
    for column in df_with_clusters:
        if column == 'Cluster':
            continue

        value_counts = cluster_df[column].value_counts()
        value_counts_str = ', '.join([f"{idx}: {cnt}" for idx, cnt in value_counts.items()])
        summary += f"{column} - {value_counts_str}\n"

    cluster_descriptions[cluster_id] = summary

In [7]:
print(cluster_descriptions["Cluster 5"])

age - 45-54: 9, 25-34: 7, 18-24: 6, 35-44: 6, 55-64: 0, <18: 0, >=65: 0, unknown: 0
edu_level - Średnie: 28, Podstawowe: 0, Wyższe: 0
fav_animals - Koty: 9, Psy: 9, Inne: 7, Brak ulubionych: 3, Koty i Psy: 0
fav_place - W górach: 18, W lesie: 7, Inne: 3, Nad wodą: 0
gender - Mężczyzna: 25, Kobieta: 3



In [9]:
prompt = "Użyliśmy algorytmu klastrowania."
for cluster_id, description in cluster_descriptions.items():
    prompt += f"\n\nKlaster {cluster_id}:\n{description}"

prompt += """
Wygeneruj najlepsze nazwy dla każdego z klasterów oraz ich opisy

Użyj formatu JSON. Przykładowo:
{
    "Cluster 0": {
        "name": "Klaster 0",
        "description": "W tym klastrze znajdują się osoby, które..."
    },
    "Cluster 1": {
        "name": "Klaster 1",
        "description": "W tym klastrze znajdują się osoby, które..."
    }
}
"""
print(prompt)

Użyliśmy algorytmu klastrowania.

Klaster Cluster 0:
age - 25-34: 12, >=65: 4, 18-24: 3, unknown: 2, <18: 1, 35-44: 0, 45-54: 0, 55-64: 0
edu_level - Wyższe: 15, Średnie: 6, Podstawowe: 1
fav_animals - Psy: 14, Koty: 4, Brak ulubionych: 3, Inne: 1, Koty i Psy: 0
fav_place - Nad wodą: 12, Inne: 1, W górach: 0, W lesie: 0
gender - Mężczyzna: 16, Kobieta: 5


Klaster Cluster 1:
age - 45-54: 18, 35-44: 8, 25-34: 7, 55-64: 4, >=65: 1, 18-24: 0, <18: 0, unknown: 0
edu_level - Wyższe: 38, Podstawowe: 0, Średnie: 0
fav_animals - Psy: 23, Brak ulubionych: 5, Inne: 5, Koty: 5, Koty i Psy: 0
fav_place - W lesie: 38, Inne: 0, Nad wodą: 0, W górach: 0
gender - Mężczyzna: 29, Kobieta: 9


Klaster Cluster 5:
age - 45-54: 9, 25-34: 7, 18-24: 6, 35-44: 6, 55-64: 0, <18: 0, >=65: 0, unknown: 0
edu_level - Średnie: 28, Podstawowe: 0, Wyższe: 0
fav_animals - Koty: 9, Psy: 9, Inne: 7, Brak ulubionych: 3, Koty i Psy: 0
fav_place - W górach: 18, W lesie: 7, Inne: 3, Nad wodą: 0
gender - Mężczyzna: 25, Kobiet

In [10]:
response = openai_client.chat.completions.create(
    model="gpt-4o",
    temperature=0,
    messages=[
        {
            "role": "user",
            "content": [{"type": "text", "text": prompt}],
        }
    ],
)

In [11]:
result = response.choices[0].message.content.replace("```json", "").replace("```", "").strip()
cluster_names_and_descriptions = json.loads(result)

In [12]:
with open("welcome_survey_cluster_names_and_descriptions_v2.json", "w") as f:
    f.write(json.dumps(cluster_names_and_descriptions))

In [13]:
with open("welcome_survey_cluster_names_and_descriptions_v2.json", "r") as f:
    print(json.loads(f.read()))

{'Cluster 0': {'name': 'Młodzi Miłośnicy Wody', 'description': 'W tym klastrze znajdują się głównie młodzi mężczyźni w wieku 25-34 lat z wyższym wykształceniem, którzy preferują spędzanie czasu nad wodą i są miłośnikami psów.'}, 'Cluster 1': {'name': 'Doświadczeni Leśni Entuzjaści', 'description': 'W tym klastrze dominują mężczyźni w wieku 45-54 lat z wyższym wykształceniem, którzy preferują spędzanie czasu w lesie i są miłośnikami psów.'}, 'Cluster 5': {'name': 'Średnio Wykształceni Górscy Odkrywcy', 'description': 'W tym klastrze znajdują się mężczyźni w wieku 25-54 lat ze średnim wykształceniem, którzy preferują spędzanie czasu w górach i mają zróżnicowane preferencje dotyczące ulubionych zwierząt.'}, 'Cluster 4': {'name': 'Wykształceni Miłośnicy Wody', 'description': 'W tym klastrze dominują mężczyźni w wieku 35-44 lat z wyższym wykształceniem, którzy preferują spędzanie czasu nad wodą i są miłośnikami psów.'}, 'Cluster 2': {'name': 'Górscy Profesjonaliści', 'description': 'W tym k